## Just using mean of fasttext embedding
https://towardsdatascience.com/super-easy-way-to-get-sentence-embedding-using-fasttext-in-python-a70f34ac5b7c

https://arxiv.org/abs/1805.09843

In [4]:
import tensorflow as tf
from tensorflow.python.keras import layers
import gensim.models.fasttext
from tensorflow.python import keras
from tensorflow.python.keras import backend as K
import re
import numpy as np
import pandas as pd
import sklearn.metrics
from nltk.corpus import stopwords

In [10]:
loaded_word_model = gensim.models.fasttext.load_facebook_vectors('../models/words_fast_text.bin')

In [58]:
# import sys
# sys.path.append("../scripts/")
from importlib import reload
import clean_text
import TweetToVect2 as TW

reload(TW)
reload(clean_text)
TW.TweetToVect

TweetToVect2.TweetToVect

In [59]:
model = TW.TweetToVect()
model.load_model(fast_text_model=loaded_word_model)
vectors = model.get_vectors([
    """
        En una entrevista radial, el presidente @alferdez afirmó que "es una discusión falsa la cantidad de miembros que podría tener la Corte Suprema de Justicia" en la reforma judicial impulsada esta semana.
    """,
    """
        Recordá que para prevenir contagios de Covid-19 es fundamental respetar los protocolos de higiene y cuidado
    """,
    """
        A quienes justifican la necesidad de la reforma judicial x la inseguridad (robos y homicidios), les digo q la reforma oficialista se refiere a juzgados federales. El robo y el homicidio NO son competencia de los juzgados federales. Buscan impunidad no luchar contra la inseguridad
    """
])
model.pairwise_similarity_vectors(vectors, vectors)

array([[1.        , 0.69555867, 0.8390938 ],
       [0.69555867, 0.9999998 , 0.6718305 ],
       [0.8390938 , 0.6718305 , 1.0000001 ]], dtype=float32)

## Test with df

In [47]:
df = pd.read_pickle('../datasets/tweets_cleaned.pkl')
df = df[(df.lang=='es')&(df.favorite_count > 10)]
tokenize = lambda text: text.split(' ')
df.loc[:,'tweet_l'] = df.full_text_cleaned.apply(lambda text: len(tokenize(text)))

In [48]:
df_sample = df[df.tweet_l > 5].sample(30000)

In [49]:
tweet_vectors = model.get_vectors(df_sample.full_text_cleaned.values, True)

In [53]:
for i in range(30):
    to_compare = df[df.tweet_l > 7].sample(1)
    to_compare_cleaned = to_compare.full_text_cleaned.values
    to_compare_text = to_compare.full_text.values
    to_compare_vector = model.get_vectors(to_compare_cleaned, True)
    similarities = model.pairwise_similarity_vectors(to_compare_vector, tweet_vectors)
    similarity_index = np.argsort(similarities[0])[::-1]
    similarity_sorted = np.sort(similarities[0])[::-1]
    
    print('SIMILAR TO:', to_compare_text[0].replace('\n', ' '))
    similar_texts = df_sample.iloc[similarity_index].head(3).full_text.apply(lambda x: x.replace('\n', ' ')).values
    for i in range(len(similar_texts)):
        print('----', round(similarity_sorted[i], 2), similar_texts[i])
    print('-----------------------------\n')

SIMILAR TO: Esta tarde, en barrio Colibrí, inauguramos el nuevo pavimento de la calle Soldado Mansilla, junto a cientos de vecinos #SanMiguel https://t.co/rqcJFazB5T
---- 1.0 Esta tarde, en barrio Colibrí, inauguramos el nuevo pavimento de la calle Soldado Mansilla, junto a cientos de vecinos #SanMiguel https://t.co/rqcJFazB5T
---- 0.93 Junto a vecinos de barrio Constantini, esta tarde inauguramos la pavimentación de la calle Mateo Bootz, muy importante para el desarrollo de la zona @BAProvincia #SanMiguel https://t.co/Uxj7MkEZzY
---- 0.92 Este jueves, en la inauguración del pavimento de Callao, tuvimos el gusto de escuchar a vecinos de barrio Parque San Miguel. Entre ellos habló Tito Baratini y nos dejó un gran testimonio #SanMiguel https://t.co/nOqYz9lspJ
-----------------------------

SIMILAR TO: Datos diarios sobre la situación del #Coronavirus en Rosario al 31 de mayo:  ➡️Ninguna persona dio positivo para #covid19 hoy. Total: 113  ➡️96 personas que se encuentran recuperadas  https

In [81]:
for i in ['🌈', '🛫', '💚', '👮‍♂️']:
    to_compare_text = [i]
    to_compare_vector = model.get_vectors(to_compare_text)
    similarities = model.pairwise_similarity_vectors(to_compare_vector, tweet_vectors)
    similarity_index = np.argsort(similarities[0])[::-1]
    similarity_sorted = np.sort(similarities[0])[::-1]
    
    print('SIMILAR TWEETS TO:', to_compare_text[0].replace('\n', ' '))
    similar_texts = df_sample.iloc[similarity_index].head(4).full_text.apply(lambda x: x.replace('\n', ' ')).values
    for i in range(len(similar_texts)):
        print('----', round(similarity_sorted[i], 2), similar_texts[i])
    print('-----------------------------\n')

SIMILAR TWEETS TO: 🌈
---- 0.94 🏳️‍🌈✨ buenas noches 🏳️‍🌈✨ https://t.co/EUEvc0pssB
---- 0.9 Felices 10 años de Matrimonio igualitario ❤🌈🌈🌈 https://t.co/6hSKdplA5K
---- 0.88 ¡Todos los días! 🏳️‍🌈  #IDAHOT https://t.co/kQ9JEFpfet
---- 0.86 El mismo amor, los mismos derechos ❤️🧡💛💚💙💜 #10AñosMatrimonioIgualitario 🏳️‍🌈 https://t.co/65tGRjyhiI
-----------------------------

SIMILAR TWEETS TO: 🛫
---- 0.71 En Ezeiza con la tripulación de Edelweiss - todo listo para el vuelo ✈️ de repatriación 👍🏼 https://t.co/kFkO0q1k8m
---- 0.64 Despegó desde Roma un nuevo vuelo especial de @Aerolineas_AR. Desde el inicio de la pandemia, programamos más de 67 operaciones a destinos internacionales que han permitido que miles de argentinos y argentinas vuelvan a sus hogares. ✈️🇮🇹🇦🇷 https://t.co/bPN0k9J7Ge
---- 0.63 Vuelo Especial a Shanghai: Listos para despegar #somosapla #pilotosargentinos #aeronauticos #flight #aerolineasargentinas #covi̇d19 https://t.co/PLxKBIrDi5
---- 0.62 Ya a bordo del Boeing 737-800 LV-ITK

Funciona bastante bien 🙄🙄